In [1]:
import numpy as np
np.random.seed(5)
#gaussian distribution sampler: normal(mean=0.0, variance_sqrt=1.0, size=None) s = np.random.normal(0,1)
#teacher model
#randomly generate z=(x,y) with x evenly sampled from (0,10) and y = 1.5 x + 5, a random noise is added. A total of N samples are drawn.
#dimension of x: d
d = 10
#number of training samples
N = 5
#dimension of hypothesis space
p = 10

#point-wise activate function f:tanh
#variance of random noise added to y
sigma = 0.1
#samples X(N*d)
X = np.random.normal(0,1,(N,d))
#random feature matrix F(d*p)
F = np.random.normal(0,1,(d,p))
#teacher parameter w(p) with lambda = ? until each dim of Y~1e0
lambda_ = 0.0001
#w_0 = np.random.normal(0,sigma/np.sqrt(lambda_*N),p)
w_0 = np.random.normal(0,1,p)
#X after the random feature matrix
X_rf = np.tanh(X.dot(F)/np.sqrt(d))
Y_pure = X_rf.dot(w_0)
Y = Y_pure + np.random.normal(0,sigma,N)
print(Y_pure)
print(Y)

[-0.55095189 -0.88060408 -1.3119232   0.78306986  0.35476296]
[-0.61558755 -0.8257473  -1.29086896  0.81772004  0.29805785]


In [2]:
#empirical risk
#L_S(w)
def L_S(w):
    diff = Y-X_rf.dot(w)
    Nloss = diff.dot(diff)
    return Nloss/N

def grad_L_S(w):
    return -2/N*X_rf.T.dot(Y-X_rf.dot(w))

def stochastic_grad_L_S(w):
    i = int(np.random.rand(1)*N)
    X_rand = X_rf[i,:]
    Y_rand = Y[i]
    return -2*X_rand.T.dot(Y_rand-X_rand.dot(w))

stochastic_grad_L_S(w_0+0.1)#when N is small, diffusion.

array([-0.05094177,  0.00796192,  0.01405325, -0.01924059,  0.05958275,
        0.02287152,  0.04894338, -0.06468644,  0.06222537, -0.0038079 ])

In [3]:
#minus log distribution: f
#prar beta(also change h if change this)
beta = 1000
#para sigma_q, can be set according to N.
sigma_q = 1
def f(w): 
    return beta*L_S(w)+(1/2/sigma_q**2)*(w.dot(w))

def grad_f(w):
    grad = beta*grad_L_S(w)
    return grad

def stochastic_grad_f(w):
    grad = stochastic_grad_L_S(w)
    return grad

In [4]:
#MCMC
#stepsize h(h*beta=0.01)
h = 0.0001
w = np.zeros(p)
for i in range(1000000):
    grad_f_w = stochastic_grad_f(w)
    proposal_state = w-h*grad_f_w+np.sqrt(2*h/beta)*np.random.normal(0,1,p)
    w = proposal_state
#print(w)
#print(w_0)
diviant = np.linalg.norm((w-w_0)/np.average(w_0))

(w-w_0)/np.average(w_0)

print(w[0:20])
print(w_0[0:20])
print(np.linalg.norm(w))
print(np.linalg.norm(w_0))
diviant

[ 0.32234051 -0.07370167 -1.26855067 -0.8626389  -0.30106368 -0.05358838
  1.35701943 -0.09685632 -0.99048833  0.26186646]
[ 1.40216662  0.46510099 -1.06503262  0.39042061  0.30560017  0.52184949
  2.23327081 -0.0347021  -1.27962318  0.03654264]
2.335965990543273
3.2345486345966656


7.264911419643334